In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')

In [ ]:
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test.head()

In [ ]:
sub = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')
sub.head()

In [ ]:
train.isna().sum()

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train, title='Pandas Profiling Report', explorative=True)
profile.to_notebook_iframe()


In [ ]:
import nltk
from nltk.corpus import stopwords 
import string 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
nlp = spacy.load('en')
lmtzr = WordNetLemmatizer()
def text_preprocess(text):
    lm = []
    text = nlp(text)
    for token in text:
        lm.append(token.lemma_)
        
    text = " ".join(lm)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
   

    
    return " ".join(text)

In [ ]:
train['processed'] = train['excerpt'].apply(text_preprocess)


In [ ]:
test['processed'] = train['excerpt'].apply(text_preprocess)

In [ ]:
train['processed1'] = train.processed.str.replace(r"[0-9]","")
test['processed1'] = test.processed.str.replace(r"[0-9]","")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(train['processed1'] , train['target'] )

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
vectorizer = TfidfVectorizer("english")
processed  = vectorizer.fit(train['processed1'] ) 
processed_test  = vectorizer.transform(test['processed1'] ) 
X_train = vectorizer.transform(X_train ) 
X_test = vectorizer.transform(X_test )


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse

from sklearn.feature_extraction.text import CountVectorizer as CV


In [ ]:
ridge = Ridge(fit_intercept = True, normalize = False)
lr = LinearRegression()

In [ ]:
X = X_train
y = y_train

In [ ]:
ridge.fit(X, y)
lr.fit(X, y)

In [ ]:
y_pred_lr = lr.predict(X_test)
y_pred_ridge = ridge.predict(X_test)

In [ ]:
MSE = mse(y_test, y_pred_lr)
MSE


In [ ]:
MSE = mse(y_test, y_pred_ridge)
MSE

In [ ]:
y_pred = ridge.predict(processed_test)


In [ ]:
#test_pred = training_all(lr,X,y)
predictions = pd.DataFrame()
predictions['id'] = test['id']
predictions['target'] = y_pred
predictions.to_csv("/kaggle/working/submission.csv", index=False)
predictions

In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
RF = RandomForestRegressor(n_estimators=100)


In [ ]:
RF.fit(X,y)


In [ ]:
y_pred=  RF.predict(processed_test)
#test_pred = training_all(lr,X,y)
predictions = pd.DataFrame()
predictions['id'] = test['id']
predictions['target'] = y_pred
predictions.to_csv("/kaggle/working/submission.csv", index=False)
predictions